In [3]:
## Laufzeit 1 min 30 sec

import pandas as pd

df = pd.read_csv('/Users/davidsanders/Documents/GitHub/masterarbeit-flex-kpis/data/epex/lastgang_lkw/lastgang_lkw_cl_2_quote_80-80-80_netz_100_pow_100-100-100_pause_45-540_M_1_Base.csv', sep=';', decimal=',')

In [6]:
df.head()

,LKW_ID,Datum,Woche,Tag,Zeit,Ladetyp,Ladestrategie,Ladezeit,DayAhead,Kosten_DayAhead,Intraday,Kosten_Intraday,Leistung,SOC,Max_Leistung,Pplus,Pminus,z
0,1,2024-01-01 00:15:00,1,1,15,NCS,DayAhead,0,0.006877,0.0,30.360,0.0,0.0,0.201897,100.0,0.0,0.0,1.0
1,1,2024-01-01 00:20:00,1,1,20,NCS,DayAhead,5,0.005965,0.0,21.892,0.0,0.0,0.201897,100.0,0.0,0.0,1.0
2,1,2024-01-01 00:25:00,1,1,25,NCS,DayAhead,10,0.005118,0.0,6.933,0.0,0.0,0.201897,100.0,0.0,0.0,1.0
3,1,2024-01-01 00:30:00,1,1,30,NCS,DayAhead,15,0.004336,0.0,-2.890,-0.0,0.0,0.201897,100.0,0.0,0.0,1.0
4,1,2024-01-01 00:35:00,1,1,35,NCS,DayAhead,20,0.003618,0.0,-5.640,-0.0,0.0,0.201897,100.0,0.0,0.0,1.0


In [12]:
df['Ladetyp'].unique()

array(['NCS', 'HPC', 'MCS'], dtype=object)

In [13]:
df['Ladestrategie'].unique()

array(['DayAhead', 'Intraday', 'Konstant', 'T_min'], dtype=object)

In [ ]:
df['Woche'].unique()